# SolsysGen & WaterIO
## Overview, Tutorial, and Demonstration

This notebook is a guided demo of:

- **solsysgen**: deterministic heliocentric systems (2D circular Keplerian orbits)
- **waterio**: checkpointing NumPy arrays to compressed `.npz`

It is suitable for:
- a 10–15 minute walkthrough,
- a reproducible tutorial.


---
## 1. Conceptual overview

### solsysgen

**SolsysGen** models a simple heliocentric system:
- circular, coplanar 2D orbits,
- explicit time stepping,
- deterministic procedural generation,
- clarity over realism.

Core pieces:
- `Sun` (central body)
- `Planet` (orbital parameters + phase)
- `SolarSystem` (container + `step()` + `state_m()`)
- Kepler helper functions (`period_s`, `circular_speed_mps`)
- JSON export/import

### waterio

**WaterIO** provides checkpoint helpers for saving/loading NumPy arrays:
- `save_checkpoint(path, **arrays)`
- `load_checkpoint(path) -> dict[str, np.ndarray]`

This is useful for storing time-series outputs like positions over time.


---
## 2. Imports and setup

Run this once to import the public APIs.


In [ ]:
from __future__ import annotations

import math
from pathlib import Path

import numpy as np

from solsysgen import SolarSystem, Sun, generate_planets
from solsysgen.constants import AU_M, DAY_S, YEAR_S
from solsysgen.io import load_json, save_json
from solsysgen.kepler import circular_speed_mps, period_s
from waterio import load_checkpoint, save_checkpoint


---
## 3. Minimal generated system

Create a Sun, generate planets deterministically, and inspect a snapshot.


In [ ]:
sun = Sun()
planets = generate_planets(sun, 5, seed=1, inner_au=0.6, outer_au=12.0)
system = SolarSystem(sun=sun, planets=planets)

len(system), [p.name for p in system.planets]


### Summary table
Distances are shown in AU, periods in days/years.


In [ ]:
rows = []
for p in system.planets:
    rows.append(
        {
            "name": p.name,
            "kind": p.kind,
            "a_AU": p.distance_m / AU_M,
            "T_days": p.period_s / DAY_S,
            "T_years": p.period_s / YEAR_S,
            "v_kms": p.orbital_speed_mps / 1000.0,
        }
    )

rows


---
## 4. Time stepping and orbit tracking

We pick one planet and track its position for a few steps.


In [ ]:
target = system.planets[len(system.planets) // 2]
target.name, target.kind, target.distance_m / AU_M


In [ ]:
steps = 12
dt = 10 * DAY_S

track = np.zeros((steps + 1, 2), dtype=np.float64)
t_days = np.zeros((steps + 1,), dtype=np.float64)

t = 0.0
for i in range(steps + 1):
    x, y = target.position_m()
    track[i, :] = (x, y)
    t_days[i] = t / DAY_S
    system.step(dt)
    t += dt

track[:3], t_days[:3]


### Sanity check: radius stays ~constant (circular orbit)


In [ ]:
r = np.sqrt(track[:, 0] ** 2 + track[:, 1] ** 2)
float(r.min()), float(r.max()), float(r.mean())


---
## 5. Kepler helpers (direct)

Compute orbital period and circular speed directly from distance and mass.
This mirrors the formulas used inside `generate_planets`.


In [ ]:
a = 1.0 * AU_M
T = period_s(a, sun.mass_kg)
v = circular_speed_mps(a, sun.mass_kg)

T / DAY_S, T / YEAR_S, v / 1000.0


---
## 6. JSON export + reload roundtrip

We export a system to JSON and reload it, verifying basic invariants.


In [ ]:
out_dir = Path("_demo_outputs")
out_dir.mkdir(exist_ok=True)

json_path = out_dir / "system.json"
save_json(json_path, system)
loaded = load_json(json_path)

(
    json_path,
    len(system.planets),
    len(loaded.planets),
    [p.kind for p in system.planets] == [p.kind for p in loaded.planets],
)


---
## 7. Checkpoint planet positions with WaterIO

Here we save a `(time, planet, xy)` position cube to `npz` and reload it.


In [ ]:
# fresh system so we start from a clean phase
sun2 = Sun()
planets2 = generate_planets(sun2, 4, seed=3, inner_au=0.7, outer_au=6.0)
system2 = SolarSystem(sun=sun2, planets=planets2)

steps = 25
dt = 5 * DAY_S

pos = np.zeros((steps, len(planets2), 2), dtype=np.float64)
for t in range(steps):
    for i, p in enumerate(system2.planets):
        pos[t, i, :] = p.position_m()
    system2.step(dt)

ckpt_path = out_dir / "positions_ckpt.npz"
save_checkpoint(ckpt_path, positions=pos)

loaded_ckpt = load_checkpoint(ckpt_path)
loaded_ckpt["positions"].shape, loaded_ckpt["positions"][0, 0]


---
## 8. Optional: Cython extension smoke check

Your repository may include an optional compiled extension.
This cell checks whether it is importable (it is OK if it is not).


In [ ]:
try:
    from solsysgen import planet as planet_ext  # type: ignore
    
    print("Cython extension available:", planet_ext)
except Exception as e:
    print("Cython extension not available (OK).")
    print("Reason:", repr(e))


---
## Summary

- **solsysgen** builds deterministic heliocentric systems with simple Keplerian circular orbits.
- You can step systems in time and inspect planet positions.
- Systems can be exported to JSON and reloaded.
- **waterio** stores NumPy checkpoint arrays to compressed `.npz` for reproducibility.
